# Dont Re-run this file just read

In [10]:
import pandas as pd

In [11]:
train = pd.read_csv("train.csv")

In [12]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [13]:
train.shape

(614, 13)

In [14]:
# We know that machine learning models take only numbers as inputs and can not process strings.
# So, we have to deal with the categories present in the dataset and convert them into numbers.

train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})


In [15]:
train.shape

(614, 13)

In [16]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,0.0,0.0,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,0.0,1.0,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,0.0,1.0,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,0.0,1.0,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,0.0,0.0,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [17]:
# Here, we have converted the categories present in the Gender, Married and the Loan Status variable into numbers,
# simply using the map function of python.Next, let’s check if there are any missing values in the dataset:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [18]:
# So, there are missing values on many variables including the Gender, Married, LoanAmount variable. 
# Next, we will remove all the rows which contain any missing values in them:
train = train.dropna()
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [19]:
# Now there are no missing values in the dataset.
# Next, we will separate the dependent (Loan_Status) and the independent variables:
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount', 'Credit_History']]
y = train.Loan_Status
X.shape, y.shape

((480, 5), (480,))

In [20]:
#  Next, let’s move on to the model building stage.Here, we will first split our dataset into a training and validation set,
# so that we can train the model on the training set and evaluate its performance on the validation set.

from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size = 0.2, random_state = 10)

# We have split the data using the train_test_split function from the sklearn library keeping the test_size as 0.2 
# which means 20 percent of the total dataset will be kept aside for the validation set.

In [21]:
# Next, we will train the random forest model using the training set:

from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(max_depth=4, random_state = 10) 
model.fit(x_train, y_train)

# Here, I have kept the max_depth as 4 for each of the trees of our random forest and 
# stored the trained model in a variable named model. 

RandomForestClassifier(max_depth=4, random_state=10)

In [22]:
# Now, our model is trained, let’s check its performance on both the training and validation set:

from sklearn.metrics import accuracy_score
pred_cv = model.predict(x_cv)
accuracy_score(y_cv,pred_cv)

# The model is 80% accurate on the validation set

0.8020833333333334

In [23]:
# Let’s check the performance on the training set too:

pred_train = model.predict(x_train)
accuracy_score(y_train,pred_train)

# Performance on the training set is almost similar to that on the validation set. So, the model has generalized well.

0.8203125

## Finally, we will save this trained model so that it can be used in the future to make predictions on new observations:

In [24]:
# saving the model 

import pickle 
pickle_out = open("classifier.pkl", mode = "wb") 
pickle.dump(model, pickle_out) 
pickle_out.close()

# We are saving the model in pickle format and storing it as classifier.pkl.
# This will store the trained model and we will use this while deploying the model.

## Model Deployment of the Loan Prediction model using Streamlit

In [25]:
!pip install -q pyngrok

!pip install -q streamlit

!pip install -q streamlit_ace

In [26]:
# We have installed 3 libraries here.
# pyngrok is a python wrapper for ngrok which helps to open secure tunnels from public URLs to localhost.
# This will help us to host our web app. Streamlit will be used to make our web app. 

In [30]:
# %%writefile app.py
 
# import pickle
# import streamlit as st
 
# # loading the trained model
# pickle_in = open('classifier.pkl', 'rb') 
# classifier = pickle.load(pickle_in)
 
# @st.cache()
  
# # defining the function which will make the prediction using the data which the user inputs 
# def prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History):   
 
#     # Pre-processing user input    
#     if Gender == "Male":
#         Gender = 0
#     else:
#         Gender = 1
 
#     if Married == "Unmarried":
#         Married = 0
#     else:
#         Married = 1
 
#     if Credit_History == "Unclear Debts":
#         Credit_History = 0
#     else:
#         Credit_History = 1  
 
#     LoanAmount = LoanAmount / 1000
 
#     # Making predictions 
#     prediction = classifier.predict( 
#         [[Gender, Married, ApplicantIncome, LoanAmount, Credit_History]])
     
#     if prediction == 0:
#         pred = 'Rejected'
#     else:
#         pred = 'Approved'
#     return pred
      
  
#     # this is the main function in which we define our webpage 

# def main():       
#     # front end elements of the web page 
#     html_temp = """ 
#     <div style ="background-color:yellow;padding:13px"> 
#     <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
#     </div> 
#     """
      
#     # display the front end aspect
#     st.markdown(html_temp, unsafe_allow_html = True) 
      
#     # following lines create boxes in which user can enter data required to make prediction 
#     Gender = st.selectbox('Gender',("Male","Female"))
#     Married = st.selectbox('Marital Status',("Unmarried","Married")) 
#     ApplicantIncome = st.number_input("Applicants monthly income") 
#     LoanAmount = st.number_input("Total loan amount")
#     Credit_History = st.selectbox('Credit_History',("Unclear Debts","No Unclear Debts"))
#     result =""
      
#     # when 'Predict' is clicked, make the prediction and store it 
#     if st.button("Predict"): 
#         result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Credit_History) 
#         st.success('Your loan is {}'.format(result))
#         print(LoanAmount)
     
# if __name__=='__main__': 
#     main()

In [32]:
# !streamlit run app.py &>/dev/null&